[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/text-to-video-synthesis-colab/blob/main/watermark_remover_gradio.ipynb)

In [ ]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q gradio==3.26.0 pytorch-lightning==1.8.3.post0 hydra-core==1.1.0 webdataset kornia==0.5.0
!git clone -b dev https://github.com/camenduru/lama
%cd /content/lama

!mkdir /content/lama/big-lama
!mkdir /content/lama/big-lama/models
!mkdir /content/lama/masks
!wget https://huggingface.co/camenduru/big-lama/resolve/main/big-lama/models/best.ckpt -O /content/lama/big-lama/models/best.ckpt
!wget https://huggingface.co/camenduru/big-lama/raw/main/big-lama/config.yaml -O /content/lama/big-lama/config.yaml
!wget https://huggingface.co/camenduru/big-lama/resolve/main/big-lama/mask.png -O /content/lama/masks/test_mask.png

!mkdir /content/lama/indir
!mkdir /content/lama/outdir
!wget https://huggingface.co/camenduru/big-lama/resolve/main/big-lama/image_mask.png -O /content/lama/indir/image_mask.png

#https://huggingface.co/spaces/fffiloni/lama-video-watermark-remover/blob/main/app.py modified

%cd /content/lama

from moviepy.editor import *
import os, cv2
import gradio as gr
from PIL import Image, ImageOps
from moviepy.editor import *

def get_frames(video_in):
    frames = []
    clip = VideoFileClip(video_in)
    if clip.fps > 30:
        print("vide rate is over 30, resetting to 30")
        clip_resized = clip.resize(height=256)
        clip_resized.write_videofile("video_resized.mp4", fps=30)
    else:
        print("video rate is OK")
        clip_resized = clip.resize(height=256)
        clip_resized.write_videofile("video_resized.mp4", fps=clip.fps)
    print("video resized to 512 height")
    cap=cv2.VideoCapture("video_resized.mp4")
    fps=cap.get(cv2.CAP_PROP_FPS)
    print("video fps: " + str(fps))
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        cv2.imwrite('in'+str(i)+'.jpg',frame)
        frames.append('in'+str(i)+'.jpg')
        i+=1
    cap.release()
    cv2.destroyAllWindows()
    print("broke the video into frames")
    return frames, fps

def create_video(frames, fps, type):
    print("building video result")
    clip = ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(type + "_result.mp4", fps=fps)
    return type + "_result.mp4"

def lama(img):
    print(img)
    img = Image.open(img)
    imageio.imwrite(f"/content/lama/indir/image.png", img)
    os.system('python predict.py model.path=/content/lama/big-lama indir=/content/lama/indir outdir=/content/lama/outdir device=cuda')
    return f"/content/lama/outdir/image_mask.png"

def infer(video_in):
    break_vid = get_frames(video_in)
    frames_list= break_vid[0]
    fps = break_vid[1]
    n_frame = len(frames_list)
    if n_frame >= len(frames_list):
        print("video is shorter than the cut value")
        n_frame = len(frames_list)
    result_frames = []
    print("set stop frames to: " + str(n_frame))
    for i in frames_list[0:int(n_frame)]:
        lama_frame = lama(i)
        lama_frame = Image.open(lama_frame)
        imageio.imwrite(f"out-{i}", lama_frame)
        result_frames.append(f"out-{i}")
        print("frame " + i + "/" + str(n_frame) + ": done;")
    final_vid = create_video(result_frames, fps, "cleaned")
    files = [final_vid]
    return final_vid

with gr.Blocks() as demo:
    with gr.Column(elem_id="col-container"):
        with gr.Row():
            with gr.Column():
                inputs = gr.Video(label="Input", source="upload", type="filepath")
            with gr.Column():
                outputs = gr.Video(label="output")
                submit_btn = gr.Button("Remove Watermark")
        submit_btn.click(infer, inputs=[inputs], outputs=[outputs])
demo.queue().launch(debug=True, share=True, inline=False)